## Gradient Descent and Linear Regression with PyTorch


#### Introduction to Linear Regression

In this tutorial, we'll discuss one of the foundational algorithms in machine learning: _Linear regression_. We'll create a model that predicts crop yields for apples and oranges (_target variables_) by looking at the average temperature, rainfall, and humidity (_input variables or features_) in a region. Here's the training data:

![linear-regression-training-data](https://i.imgur.com/6Ujttb4.png)
![image.png](attachment:image.png)
In a linear regression model, each target variable is estimated to be a weighted sum of the input variables, offset by some constant, known as a bias :

```
yield_apple  = w11 * temp + w12 * rainfall + w13 * humidity + b1
yield_orange = w21 * temp + w22 * rainfall + w23 * humidity + b2
```


In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import DataLoader
from torchvision import datasets,transforms

import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
%matplotlib inline

## Training data


In [2]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70]], dtype='float32')

In [3]:
# Targets (apples, oranges)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119]], dtype='float32')

In [4]:
# Convert inputs and targets to tensors
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)
print(inputs)
print(targets)

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]])
tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


# Linear regression model from scratch


In [5]:
# Weights and biases
w = torch.randn(2, 3, requires_grad=True)
b = torch.randn(2, requires_grad=True)
print(w)
print(b)

tensor([[-1.0757, -1.1371,  1.3976],
        [ 1.3640, -0.1145, -2.1712]], requires_grad=True)
tensor([-0.2810,  0.5265], requires_grad=True)


In [6]:
def model(x):
    return x @ w.t() + b

In [7]:
# Generate predictions
preds = model(inputs)
print(preds)

tensor([[ -94.8973,   -0.9361],
        [-108.7902,  -24.3846],
        [-165.1796,  -22.0819],
        [-107.1872,   54.3968],
        [ -85.8365,  -68.3367]], grad_fn=<AddBackward0>)


In [8]:
# Compare with targets
print(targets)

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


## Loss function


In [9]:
# MSE loss
def mse(t1, t2):
    diff = t1 - t2
    return torch.sum(diff * diff) / diff.numel()

In [10]:
# Compute loss
loss = mse(preds, targets)
print(loss)

tensor(27209.8242, grad_fn=<DivBackward0>)


## Compute gradients


In [11]:
# Compute gradients
loss.backward()

In [12]:
# Gradients for weights
print(w)
print(w.grad)

tensor([[-1.0757, -1.1371,  1.3976],
        [ 1.3640, -0.1145, -2.1712]], requires_grad=True)
tensor([[-15843.3691, -17715.0156, -10623.2109],
        [ -8246.4453, -10760.7617,  -6507.9014]])


## Adjust weights and biases to reduce the loss


In [13]:
w
w.grad

tensor([[-15843.3691, -17715.0156, -10623.2109],
        [ -8246.4453, -10760.7617,  -6507.9014]])

In [14]:
with torch.no_grad():
    w -= w.grad * 1e-5
    b -= b.grad * 1e-5

In [15]:
# Let's verify that the loss is actually lower
loss = mse(preds, targets)
print(loss)

tensor(27209.8242, grad_fn=<DivBackward0>)


In [16]:
w.grad.zero_()
b.grad.zero_()
print(w.grad)
print(b.grad)

tensor([[0., 0., 0.],
        [0., 0., 0.]])
tensor([0., 0.])


# Train the model using gradient descent


In [17]:
# Generate predictions
preds = model(inputs)
print(preds)

tensor([[ -66.8927,   15.0929],
        [ -71.9828,   -3.2447],
        [-121.4944,    3.2875],
        [ -79.4770,   69.8442],
        [ -50.4600,  -47.7598]], grad_fn=<AddBackward0>)


In [18]:
# Calculate the loss
loss = mse(preds, targets)
print(loss)

tensor(18978.6133, grad_fn=<DivBackward0>)


In [19]:
# Compute gradients
loss.backward()
print(w.grad)
print(b.grad)

tensor([[-12951.0020, -14603.6445,  -8704.1621],
        [ -6587.1582,  -8966.0840,  -5402.7881]])
tensor([-154.2614,  -84.5560])


In [20]:
# Adjust weights & reset gradients
with torch.no_grad():
    w -= w.grad * 1e-5
    b -= b.grad * 1e-5
    w.grad.zero_()
    b.grad.zero_()

In [21]:
print(w)
print(b)

tensor([[-0.7877, -0.8139,  1.5908],
        [ 1.5124,  0.0827, -2.0521]], requires_grad=True)
tensor([-0.2776,  0.5284], requires_grad=True)


In [22]:
# Calculate loss
preds = model(inputs)
loss = mse(preds, targets)
print(loss)

tensor(13425.2812, grad_fn=<DivBackward0>)


## Train for multiple epochs


In [23]:
# Train for 100 epochs
for i in range(100):
    preds = model(inputs)
    loss = mse(preds, targets)
    loss.backward()
    with torch.no_grad():
        w -= w.grad * 1e-5
        b -= b.grad * 1e-5
        w.grad.zero_()
        b.grad.zero_()

In [24]:
# Calculate loss
preds = model(inputs)
loss = mse(preds, targets)
print(loss)

tensor(821.7184, grad_fn=<DivBackward0>)


In [25]:
# Predictions
preds

tensor([[ 59.0271,  81.0991],
        [ 93.8823,  89.6898],
        [ 89.1453, 140.5863],
        [ 33.2183,  97.3737],
        [115.0954,  64.9170]], grad_fn=<AddBackward0>)

In [26]:
# Targets
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])

# Linear regression using PyTorch built-ins


In [27]:
import torch.nn as nn

In [28]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70], 
                   [74, 66, 43], 
                   [91, 87, 65], 
                   [88, 134, 59], 
                   [101, 44, 37], 
                   [68, 96, 71], 
                   [73, 66, 44], 
                   [92, 87, 64], 
                   [87, 135, 57], 
                   [103, 43, 36], 
                   [68, 97, 70]], 
                  dtype='float32')

# Targets (apples, oranges)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119],
                    [57, 69], 
                    [80, 102], 
                    [118, 132], 
                    [21, 38], 
                    [104, 118], 
                    [57, 69], 
                    [82, 100], 
                    [118, 134], 
                    [20, 38], 
                    [102, 120]], 
                   dtype='float32')

inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

In [29]:
inputs

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.],
        [ 74.,  66.,  43.],
        [ 91.,  87.,  65.],
        [ 88., 134.,  59.],
        [101.,  44.,  37.],
        [ 68.,  96.,  71.],
        [ 73.,  66.,  44.],
        [ 92.,  87.,  64.],
        [ 87., 135.,  57.],
        [103.,  43.,  36.],
        [ 68.,  97.,  70.]])

In [30]:
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 57.,  69.],
        [ 80., 102.],
        [118., 132.],
        [ 21.,  38.],
        [104., 118.],
        [ 57.,  69.],
        [ 82., 100.],
        [118., 134.],
        [ 20.,  38.],
        [102., 120.]])

## Dataset and DataLoader


In [31]:
from torch.utils.data import TensorDataset

In [32]:
# Define dataset
train_ds = TensorDataset(inputs, targets)
train_ds[0:3]

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.]]),
 tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.]]))

In [33]:
from torch.utils.data import DataLoader

In [34]:
# Define data loader
batch_size = 5
train_dl = DataLoader(train_ds, batch_size, shuffle=True)

In [35]:
for xb, yb in train_dl:
    print(xb)
    print(yb)
    break

tensor([[101.,  44.,  37.],
        [ 73.,  67.,  43.],
        [ 91.,  87.,  65.],
        [ 68.,  96.,  71.],
        [ 69.,  96.,  70.]])
tensor([[ 21.,  38.],
        [ 56.,  70.],
        [ 80., 102.],
        [104., 118.],
        [103., 119.]])


## nn.Linear


In [36]:
# Define model
model = nn.Linear(3, 2)
print(model.weight)
print(model.bias)

Parameter containing:
tensor([[-0.0327, -0.4510, -0.1730],
        [-0.1625, -0.1868, -0.1903]], requires_grad=True)
Parameter containing:
tensor([-0.1484,  0.3703], requires_grad=True)


In [37]:
# Parameters
list(model.parameters())

[Parameter containing:
 tensor([[-0.0327, -0.4510, -0.1730],
         [-0.1625, -0.1868, -0.1903]], requires_grad=True),
 Parameter containing:
 tensor([-0.1484,  0.3703], requires_grad=True)]

In [38]:
# Generate predictions
preds = model(inputs)
preds

tensor([[-40.1895, -32.1909],
        [-53.8815, -43.0350],
        [-73.4568, -49.8349],
        [-29.2766, -31.2798],
        [-57.8079, -42.0954],
        [-39.7712, -32.1667],
        [-53.6036, -43.0385],
        [-73.6625, -50.1877],
        [-29.6949, -31.3040],
        [-57.9482, -42.1232],
        [-39.9115, -32.1945],
        [-53.4633, -43.0107],
        [-73.7348, -49.8313],
        [-29.1363, -31.2520],
        [-58.2262, -42.1197]], grad_fn=<AddmmBackward0>)

## Loss Function


In [39]:
# Import nn.functional
import torch.nn.functional as F

In [40]:
# Define loss function
loss_fn = F.mse_loss

In [41]:
loss = loss_fn(model(inputs), targets)
print(loss)

tensor(18800.4395, grad_fn=<MseLossBackward0>)


## Optimizer


In [42]:
# Define optimizer
opt = torch.optim.SGD(model.parameters(), lr=1e-5)

## Train the model


In [43]:
# Utility function to train the model
def fit(num_epochs, model, loss_fn, opt, train_dl):
    
    # Repeat for given number of epochs
    for epoch in range(num_epochs):
        
        # Train with batches of data
        for xb,yb in train_dl:
            
            # 1. Generate predictions
            pred = model(xb)
            
            # 2. Calculate loss
            loss = loss_fn(pred, yb)
            
            # 3. Compute gradients
            loss.backward()
            
            # 4. Update parameters using gradients
            opt.step()
            
            # 5. Reset the gradients to zero
            opt.zero_grad()
        
        # Print the progress
        if (epoch+1) % 10 == 0:
            print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

In [44]:
fit(100, model, loss_fn, opt, train_dl)

Epoch [10/100], Loss: 776.4268
Epoch [20/100], Loss: 211.7298
Epoch [30/100], Loss: 108.7751
Epoch [40/100], Loss: 131.7932
Epoch [50/100], Loss: 168.0137
Epoch [60/100], Loss: 78.1172
Epoch [70/100], Loss: 90.7267
Epoch [80/100], Loss: 47.9560
Epoch [90/100], Loss: 47.7763
Epoch [100/100], Loss: 30.0690


In [45]:
# Generate predictions
preds = model(inputs)
preds

tensor([[ 58.6175,  71.8288],
        [ 81.1974,  98.0216],
        [118.1341, 136.6844],
        [ 29.8771,  44.9994],
        [ 95.0402, 109.9343],
        [ 57.5680,  70.8416],
        [ 80.7750,  97.5734],
        [118.3403, 137.0282],
        [ 30.9266,  45.9866],
        [ 95.6673, 110.4734],
        [ 58.1951,  71.3806],
        [ 80.1479,  97.0344],
        [118.5565, 137.1327],
        [ 29.2499,  44.4604],
        [ 96.0897, 110.9215]], grad_fn=<AddmmBackward0>)

In [46]:
# Compare with targets
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 57.,  69.],
        [ 80., 102.],
        [118., 132.],
        [ 21.,  38.],
        [104., 118.],
        [ 57.,  69.],
        [ 82., 100.],
        [118., 134.],
        [ 20.,  38.],
        [102., 120.]])

In [47]:
model(torch.tensor([[75, 63, 44.]]))

tensor([[55.2571, 68.5166]], grad_fn=<AddmmBackward0>)